In [2]:
import pandas as pd

In [174]:
popular_lots = pd.read_excel("data/processed/recommended_popular_non_active_bidders_test.xlsx")
cf = pd.read_excel("data/processed/recommended_CF_test.xlsx")
one_to_one = pd.read_excel("data/processed/recommended_onetoone_test.xlsx")

In [147]:
popular_lots.head()

,mbr_nbr,lot_make_cd,grp_model,recommended_lot_nbr,matched_lot_make_cd,matched_grp_model,distance
0,1,CHEV,MALIBU,69134305,CHEV,MALIBU,480.94
1,1,CHEV,SILVERADO,70316735,CHEV,SILVERADO,818.53
2,1,CHEV,EQUINOX,70869825,CHEV,EQUINOX,203.97
3,1,CHEV,CRUZE,81004805,CHEV,CRUZE,710.48
4,1,FORD,ESCAPE,70107145,FORD,ESCAPE,398.49


In [148]:
popular_lots[popular_lots['mbr_nbr']==874]

,mbr_nbr,lot_make_cd,grp_model,recommended_lot_nbr,matched_lot_make_cd,matched_grp_model,distance


In [175]:
cf[cf['input_buyer_nbr']==1345]

,input_buyer_nbr,original_lot,recommended_lot,manhattan_distance
84003,1345,62470185,81016625,78.79
84007,1345,64440125,71722825,913.94
84008,1345,64292675,83982235,7651.55
84010,1345,70838555,68118115,3334.60
84011,1345,59693655,68848965,1025.65
84012,1345,66643135,83939205,92.37


In [150]:
one_to_one.head()

,input_buyer_nbr,original_lot,recommended_lot,manhattan_distance,source
0,165,55854275,70349275,3204.0,Step 1 - YMM/Manhattan
1,165,67057705,84160775,0.0,Step 1 - YMM/Manhattan
2,165,55158715,81290275,3085.0,Step 1 - YMM/Manhattan
3,165,53321025,81066205,486.0,Step 1 - YMM/Manhattan
4,165,67057705,80544035,0.0,Step 2 - Recent YMM/Manhattan


In [151]:
one_to_one['input_buyer_nbr'].value_counts()

input_buyer_nbr
165       6
691991    6
692337    6
692447    6
692409    6
         ..
377739    6
377385    6
377641    6
377119    6
999833    6
Name: count, Length: 23738, dtype: int64

In [152]:
# Step 1: Standardize column names and tag source
cf_std = cf[['input_buyer_nbr', 'recommended_lot']].copy()
cf_std['identifier'] = 1
cf_std['group'] = 'test'

one_to_one_std = one_to_one[['input_buyer_nbr', 'recommended_lot']].copy()
one_to_one_std['identifier'] = 2
one_to_one_std['group'] = 'test'

popular_std = popular_lots[['mbr_nbr', 'recommended_lot_nbr']].copy()
popular_std.rename(columns={
    'mbr_nbr': 'input_buyer_nbr',
    'recommended_lot_nbr': 'recommended_lot'
}, inplace=True)
popular_std['identifier'] = 3
popular_std['group'] = 'test'

# Step 2: Concatenate all rows (keep ALL duplicates)
combined = pd.concat([cf_std, one_to_one_std, popular_std], ignore_index=True)

# Step 3: Assign rank per buyer and keep only first 6 rows per buyer
combined['rank'] = combined.groupby('input_buyer_nbr').cumcount() + 1
combined = combined[combined['rank'] <= 6]

# Step 4: Final column order
combined_recos = combined[['identifier', 'group', 'input_buyer_nbr', 'recommended_lot']]


In [153]:
combined_recos

,identifier,group,input_buyer_nbr,recommended_lot
0,1,test,509689,85002555
1,1,test,749677,71861465
2,1,test,514361,71436405
3,1,test,748155,64487405
4,1,test,439735,55338375
...,...,...,...,...
679147,3,test,999653,71105655
679148,3,test,999653,84936185
679149,3,test,999653,71165685
679150,3,test,999653,71700525


In [154]:
combined_recos['input_buyer_nbr'].value_counts()

input_buyer_nbr
509689    6
505185    6
505397    6
505393    6
505355    6
         ..
846949    6
846907    6
846893    6
846855    6
999653    6
Name: count, Length: 111588, dtype: int64

In [155]:
# Step 1: Add rank per buyer to create lot_1 to lot_6 columns
combined_recos['rank'] = combined_recos.groupby('input_buyer_nbr').cumcount() + 1

# Step 2: Pivot to wide format
pivoted = combined_recos.pivot(index=['identifier', 'group', 'input_buyer_nbr'], columns='rank', values='recommended_lot')

# Step 3: Rename columns
pivoted.columns = [f'lot_{i}' for i in pivoted.columns]

# Step 4: Reset index to flatten the DataFrame
pivoted = pivoted.reset_index()

# Step 5: Convert lot columns to integer (optional, cleaner output)
lot_cols = [col for col in pivoted.columns if col.startswith('lot_')]
pivoted[lot_cols] = pivoted[lot_cols].fillna(0).astype(int)


In [156]:
pivoted

,identifier,group,input_buyer_nbr,lot_1,lot_2,lot_3,lot_4,lot_5,lot_6
0,1,test,75,80445465,81552565,70145775,84052465,80743485,68163505
1,1,test,107,68785905,80145435,67685275,84063895,80260515,80407165
2,1,test,153,71736895,80929605,80877995,85678625,70467545,83957545
3,1,test,223,69215145,81040225,70262465,70153315,81962705,60679065
4,1,test,295,67796175,71637875,81932845,71462085,81163665,70804025
...,...,...,...,...,...,...,...,...,...
111583,3,test,996325,69343055,54846595,69662125,85155695,80276245,67224545
111584,3,test,996431,71090815,67931235,81676375,85137435,64178715,85277865
111585,3,test,997061,81595045,68613715,84073805,69762065,80547255,62024665
111586,3,test,998511,82126485,71953865,67931235,69071455,70101295,63640125


In [157]:
pivoted['input_buyer_nbr'].nunique()

111588

In [176]:
pivoted['group'].value_counts()

group
test    111588
Name: count, dtype: int64

### Reco for holdout

In [158]:
holdout_group = pd.read_csv("data/processed/recommended_holdout.csv")

In [159]:
holdout_group.head()

,identifier,group,input_buyer_nbr,lot_1,lot_2,lot_3,lot_4,lot_5,lot_6
0,3,holdout,4,71844035,71058985,68747425,70116365,71526645,81106795
1,1,holdout,28,78445034,80804005,70993805,70869825,67410435,66995945
2,3,holdout,32,81211285,83784195,69472435,71107345,69781555,70676015
3,3,holdout,36,71539915,71619965,66929695,81411375,70951445,66323525
4,3,holdout,44,68881465,68675045,85568905,81337525,71908335,84210025


In [160]:
holdout_group[holdout_group['input_buyer_nbr']==874]

,identifier,group,input_buyer_nbr,lot_1,lot_2,lot_3,lot_4,lot_5,lot_6
115,1,holdout,874,85719615,70981435,68168125,67486995,69415325,46709085


In [161]:
# Step 1: Ensure same column order
common_cols = ['identifier', 'group', 'input_buyer_nbr', 'lot_1', 'lot_2', 'lot_3', 'lot_4', 'lot_5', 'lot_6']

# Step 2: Align both DataFrames to the same column schema
pivoted_aligned = pivoted[common_cols]
holdout_group_aligned = holdout_group[common_cols]

# Step 3: Concatenate vertically
final_df = pd.concat([pivoted, holdout_group_aligned], ignore_index=True)


In [162]:
pivoted_aligned

,identifier,group,input_buyer_nbr,lot_1,lot_2,lot_3,lot_4,lot_5,lot_6
0,1,test,75,80445465,81552565,70145775,84052465,80743485,68163505
1,1,test,107,68785905,80145435,67685275,84063895,80260515,80407165
2,1,test,153,71736895,80929605,80877995,85678625,70467545,83957545
3,1,test,223,69215145,81040225,70262465,70153315,81962705,60679065
4,1,test,295,67796175,71637875,81932845,71462085,81163665,70804025
...,...,...,...,...,...,...,...,...,...
111583,3,test,996325,69343055,54846595,69662125,85155695,80276245,67224545
111584,3,test,996431,71090815,67931235,81676375,85137435,64178715,85277865
111585,3,test,997061,81595045,68613715,84073805,69762065,80547255,62024665
111586,3,test,998511,82126485,71953865,67931235,69071455,70101295,63640125


In [163]:
final_df['input_buyer_nbr'].nunique()

223112

In [164]:
final_df.isnull().sum()

identifier         0
group              0
input_buyer_nbr    0
lot_1              0
lot_2              0
lot_3              0
lot_4              0
lot_5              0
lot_6              0
dtype: int64

In [165]:
final_df

,identifier,group,input_buyer_nbr,lot_1,lot_2,lot_3,lot_4,lot_5,lot_6
0,1,test,75,80445465,81552565,70145775,84052465,80743485,68163505
1,1,test,107,68785905,80145435,67685275,84063895,80260515,80407165
2,1,test,153,71736895,80929605,80877995,85678625,70467545,83957545
3,1,test,223,69215145,81040225,70262465,70153315,81962705,60679065
4,1,test,295,67796175,71637875,81932845,71462085,81163665,70804025
...,...,...,...,...,...,...,...,...,...
223107,2,holdout,998374,80868655,69634775,69029225,70566225,80202135,71515995
223108,2,holdout,998724,84928645,65540865,70722055,71499235,65234175,81488665
223109,3,holdout,999222,66262835,66924225,71794775,81587495,68726465,71716485
223110,3,holdout,999662,71090815,67931235,81676375,85137435,64178715,85277865


In [171]:
final_df.to_excel('data/recos_v2/recommendations_final.xlsx',index=False)

In [170]:
final_df['input_buyer_nbr'].nunique()

223112

In [172]:
final_df[final_df['input_buyer_nbr']==1345]

,identifier,group,input_buyer_nbr,lot_1,lot_2,lot_3,lot_4,lot_5,lot_6
24,1,test,1345,81016625,71722825,83982235,68118115,68848965,83939205


In [177]:
final_df['group'].value_counts()

group
test       111588
holdout    111524
Name: count, dtype: int64

#### Would Have

In [88]:
cf_wh = pd.read_excel("data/would_have/data_high_holdout_reco.xlsx")
one_to_one_wh = pd.read_excel("data/would_have/recommended_onetoone_holdout.xlsx")
popular_lots_wh = pd.read_excel("data/would_have/recommended_popular_non_active_bidders_test.xlsx")

In [101]:
cf_wh.head(2)

,input_buyer_nbr,original_lot,recommended_lot,manhattan_distance
0,63980,66920335,62633815,2410.3
1,18268,61213015,85009405,0.0


In [102]:
one_to_one_wh.head(2)

,input_buyer_nbr,original_lot,recommended_lot,manhattan_distance,source
0,298,67668425,81105395,5379.4,Step 1 - YMM/Manhattan
1,298,67668425,69996535,8650.7,Step 2 - Recent YMM/Manhattan


In [103]:
popular_lots_wh.head(2)

,identifier,group,input_buyer_nbr,lot_1,lot_2,lot_3,lot_4,lot_5,lot_6
0,3,holdout,4,71844035,71058985,68747425,70116365,71526645,81106795
1,3,holdout,32,81211285,83784195,69472435,71107345,69781555,70676015


In [99]:
popular_lots_wh['input_buyer_nbr'].nunique()

71118

In [104]:
# Step 1: Standardize column names and tag source
cf_std = cf_wh[['input_buyer_nbr', 'recommended_lot']].copy()
cf_std['identifier'] = 1
cf_std['group'] = 'holdout'

one_to_one_std = one_to_one_wh[['input_buyer_nbr', 'recommended_lot']].copy()
one_to_one_std['identifier'] = 2
one_to_one_std['group'] = 'holdout'

# Step 2: Concatenate all rows (keep ALL duplicates)
combined = pd.concat([cf_std, one_to_one_std], ignore_index=True)

# Step 3: Assign rank per buyer and keep only first 6 rows per buyer
combined['rank'] = combined.groupby('input_buyer_nbr').cumcount() + 1
combined = combined[combined['rank'] <= 6]

# Step 4: Final column order
combined_recos = combined[['identifier', 'group', 'input_buyer_nbr', 'recommended_lot']]


In [105]:
combined_recos

,identifier,group,input_buyer_nbr,recommended_lot
0,1,holdout,63980,62633815
1,1,holdout,18268,85009405
2,1,holdout,806604,70232365
3,1,holdout,806604,80335885
4,1,holdout,806604,67790155
...,...,...,...,...
243691,2,holdout,995336,43236485
243692,2,holdout,995336,83889295
243693,2,holdout,995336,81387575
243694,2,holdout,995336,85133335


In [106]:
# Step 1: Unpivot popular_lots_wh (lot_1 to lot_6 → one column)
popular_long = popular_lots_wh.melt(
    id_vars=['identifier', 'group', 'input_buyer_nbr'],
    value_vars=['lot_1', 'lot_2', 'lot_3', 'lot_4', 'lot_5', 'lot_6'],
    var_name='lot_rank',
    value_name='recommended_lot'
)
# Step 3: Ensure final column order matches combined_recos
popular_long = popular_long[['identifier', 'group', 'input_buyer_nbr', 'recommended_lot']]

# Step 4: Concatenate
final_combined_recos = pd.concat([combined_recos, popular_long], ignore_index=True)


In [107]:
final_combined_recos

,identifier,group,input_buyer_nbr,recommended_lot
0,1,holdout,63980,62633815
1,1,holdout,18268,85009405
2,1,holdout,806604,70232365
3,1,holdout,806604,80335885
4,1,holdout,806604,67790155
...,...,...,...,...
670399,3,holdout,997866,80653265
670400,3,holdout,998042,64622255
670401,3,holdout,999222,71716485
670402,3,holdout,999662,85277865


In [108]:
final_combined_recos['input_buyer_nbr'].nunique()

111734

In [109]:
# Step 1: Add rank per buyer
final_combined_recos['rank'] = final_combined_recos.groupby('input_buyer_nbr').cumcount() + 1

# Step 2: Pivot to wide format
pivoted_final = final_combined_recos.pivot(
    index=['identifier', 'group', 'input_buyer_nbr'],
    columns='rank',
    values='recommended_lot'
)

# Step 3: Rename columns
pivoted_final.columns = [f'lot_{i}' for i in pivoted_final.columns]

# Step 4: Reset index
pivoted_final = pivoted_final.reset_index()

# Step 5: Convert lots to integer for cleaner output
lot_cols = [col for col in pivoted_final.columns if col.startswith('lot_')]
pivoted_final[lot_cols] = pivoted_final[lot_cols].fillna(0).astype(int)


In [110]:
pivoted_final

,identifier,group,input_buyer_nbr,lot_1,lot_2,lot_3,lot_4,lot_5,lot_6
0,1,holdout,28,68956505,69233045,81606055,62576675,70871345,70216755
1,1,holdout,86,67183935,71103395,71968875,80316745,71258285,81257865
2,1,holdout,390,84750345,86318135,63342435,70570425,69319725,80049965
3,1,holdout,408,67924415,66323525,81659085,80347215,82168325,80693265
4,1,holdout,436,71250935,80745265,86106475,50193985,69999715,58673765
...,...,...,...,...,...,...,...,...,...
111729,3,holdout,997866,81310215,69115905,69314295,80196785,69968725,80653265
111730,3,holdout,998042,70699875,71411575,80532215,52292585,81106575,64622255
111731,3,holdout,999222,66262835,66924225,71794775,81587495,68726465,71716485
111732,3,holdout,999662,71090815,67931235,81676375,85137435,64178715,85277865


In [114]:
pivoted_final['input_buyer_nbr'].duplicated().sum()

np.int64(0)

In [111]:
pivoted_final.isnull().sum()

identifier         0
group              0
input_buyer_nbr    0
lot_1              0
lot_2              0
lot_3              0
lot_4              0
lot_5              0
lot_6              0
dtype: int64

In [178]:
pivoted_final['group'].value_counts()

group
holdout    111734
Name: count, dtype: int64

In [179]:
pivoted_final.shape

(111734, 9)

In [181]:
pivoted_final.to_excel('data/recos_v2/recommendations_holdout_would_have.xlsx',index=False)